# **Especialização em Ciência de Dados - INF/UFRGS E SERPRO**
### Disciplina CD003 - Aprendizado Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---



<br>

## **Aula 07** - **Tópico: Aprendizado Ensemble**

<br>

A teoria da **Sabedoria das multidões** estabelece que quando agregamos opiniões diversas e independentes na solução de um problema, temos grandes chances de encontrar uma solução que, no geral, é melhor do que as soluções individuais. Em Aprendizado de Máquina, esta teoria é explorada através do conceito de **aprendizado ensemble**, no qual múltiplos modelos são treinados para um mesmo conjunto de dados e suas saídas são agregadas para a tomada de decisão. Um aspecto chave no ensemble é a diversidade entre os modelos. Formas usuais de treinar um ensemble de modelos são: i) usar diferentes algoritmos e ii) usar dados de treinamento diferentes (amostras diferentes, por exemplo).

O sklearn possui um [módulo](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) com diversos métodos de aprendizado ensemble para tarefas de classificação e regressão, alguns dos quais iremos utilizar neste notebook.

**Objetivo deste notebook**: Explorar diferentes estratégias para construir *ensembles* de classificadores.


*Observação: O desenvolvimento deste notebook foi baseado no livro de Aurélien Géron, Hands-On Machine Learning with Scikit-Learn, Keras and Tensorflow (2nd Edition)*

----


###Carregando e dividindo os dados


Neste notebook, vamos utilizar novamente o dataset [Breast Cancer Wisconsin](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset), carregando-o através das funções do scikit-learn.

In [ ]:
import pandas as pd             # biblioteca para análise de dados 
import matplotlib.pyplot as plt # biblioteca para visualização de informações
import seaborn as sns           # biblioteca para visualização de informações
import numpy as np              # biblioteca para operações com arrays multidimensionais
from sklearn.datasets import load_breast_cancer ## conjunto de dados a ser analisado
sns.set()

data = load_breast_cancer() ## carrega os dados de breast cancer
X = data.data  # matriz contendo os atributos
y = data.target  # vetor contendo a classe (0 para maligno e 1 para benigno) de cada instância
feature_names = data.feature_names  # nome de cada atributo
target_names = data.target_names  # nome de cada classe

## Relembrando as características do dataset
print(f"Dimensões de X: {X.shape}\n")
print(f"Dimensões de y: {y.shape}\n")
print(f"Nomes dos atributos: {feature_names}\n")
print(f"Nomes das classes: {target_names}")

Iniciamos fazendo uma divisão dos dados em treino e teste na proporção 80%/20%. Fazemos de forma estratificada, isto é, mantendo a proporção original das classes ('stratify=y').

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 

## Fazemos a divisão com 2-way holdout, de forma estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

Fazemos a normalização dos dados, para evitar o impacto em algoritmos sensíveis a diferenças de escala entre atributos.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Bagging

O método Bagging (Boostrap Aggregating) é baseado no treinamento de múltiplos modelos usando o mesmo algoritmo de aprendizado mas diferentes dados de treinamento obtidos através de amostragem com reposição. Embora seja possível realizar estas amostragem manualmente, o scikit-learn oferece um método que constrói este tipo de ensemble: [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier) . Este método tem como principais parâmetros:


*   base_estimator: o algoritmo usado para treinamento dos modelos individuais. Por padrão, utiliza o DecisionTreeClassifier.
*   n_estimators: o número de modelos do ensemble. Por padrão, utiliza 10.
*   bootstrap: define se a amostragem deve ser feita com reposição. Por padrão, tem valor True. 
*   max_samples: o número de instâncias de cada amostra. Por padrão, é definida como o número de instâncias no conjunto de dados original.



In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

## Treina um classificador Bagging usando uma árvore de decisão como base
## São treinadas 501 árvores, cada qual com base em 50% dos dados de treinamento
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=501, max_samples=0.5, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay

y_pred_bag = bag_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred_bag,labels=bag_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=bag_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print(round(accuracy_score(y_test, y_pred_bag),3))

O sklearn possui uma versão de Bagging para regressão: [BaggingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor)


### Random Forests

Como vimos, o Random Forests (Florestas Aleatórias) é um algoritmo muito semelhante ao Bagging. A diferença é que ele introduz uma aleatoriedade extra ao construir as árvores de decisão: ao invés de escolher o melhor dentre todos os atributos para uma nova divisão de nós, o algoritmo escolhe o melhor dentre um subconjunto aleatório de atributos. No sklearn, este tipo de ensemble é implementado no método [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier). Além dos hiperparâmetros comuns a árvores de decisão (como criterion, max_depth, min_samples_split, min_samples_leaf, max_leaf_nodes, etc.), este método tem como principais parâmetros:

*   n_estimators: o número de modelos do ensemble. Por padrão, utiliza 100.
*   bootstrap: define se a amostragem deve ser feita com reposição. Por padrão, tem valor True. 
*   max_samples: o número de instâncias de cada amostra. Por padrão, é definida como o número de instâncias no conjunto de dados original.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

## Treina um classificador Random Forests.
## São treinadas 501 árvores, usando pré-poda (profundidade máxima=5)
rf_clf = RandomForestClassifier(n_estimators=501, max_depth=5, random_state=42)
rf_clf.fit(X_train, y_train)

In [ ]:

y_pred_rf = rf_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred_rf,labels=rf_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print(round(accuracy_score(y_test, y_pred_rf),3))

O sklearn possui uma versão de Random Forests para regressão: [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)


### Árvores de Decisão vs Bagging vs Random Forests

Apenas para fins de visualização, visando observar como os diferentes métodos baseados em Bagging podem encontrar fronteiras de decisão mais complexas que uma árvore de decisão (algoritmo usando como 'base' no ensemble), vamos usar um conjunto de dados sintético para treinar modelos baseados nos três algoritmos.

In [ ]:
from sklearn.datasets import make_moons
X_moon, y_moon = make_moons(n_samples=500, noise=0.30, random_state=42)
X_moon_train, X_moon_test, y_moon_train, y_moon_test = train_test_split(X_moon, y_moon, random_state=42,test_size=0.25)


In [ ]:
tree_moon = DecisionTreeClassifier(random_state=42)
tree_moon.fit(X_moon_train, y_moon_train)

bag_moon = BaggingClassifier(DecisionTreeClassifier(), n_estimators=501,
                            max_samples=100, n_jobs=-1, random_state=42)
bag_moon.fit(X_moon_train, y_moon_train)


rf_moon = RandomForestClassifier(n_estimators=501, max_depth=5, random_state=42)
rf_moon.fit(X_moon_train, y_moon_train)


## Função para plotar as fronteiras de decisão
def plot_decision_boundary(clf, X, y, alpha=1.0):
    axes=[-1.5, 2.4, -1, 1.5]
    x1, x2 = np.meshgrid(np.linspace(axes[0], axes[1], 100),
                         np.linspace(axes[2], axes[3], 100))
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    
    plt.contourf(x1, x2, y_pred, alpha=0.3 * alpha, cmap='Wistia')
    plt.contour(x1, x2, y_pred, cmap="Greys", alpha=0.8 * alpha)
    colors = ["#78785c", "#c47b27"]
    markers = ("o", "^")
    for idx in (0, 1):
        plt.plot(X[:, 0][y == idx], X[:, 1][y == idx],
                 color=colors[idx], marker=markers[idx], linestyle="none")
    plt.axis(axes)
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$", rotation=0)

## Chamanda a função para cada modelo, e gerando a figura
fig, axes = plt.subplots(ncols=3, figsize=(18, 6), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_moon,X_moon_train, y_moon_train)
plt.title("Decision Tree")
plt.sca(axes[1])
plot_decision_boundary(bag_moon,X_moon_train, y_moon_train)
plt.title("Decision Trees with Bagging")
plt.sca(axes[2])
plot_decision_boundary(rf_moon,X_moon_train, y_moon_train)
plt.title("Random Forests")
plt.ylabel("")
#save_fig("decision_tree_vs_ensembles")
plt.show()

### Boosting - Adaboost

Boosting são métodos ensemble capazes de combinar modelos fracos (isto é, que possuem um alto viés) em um modelo mais forte. A ideia é treinar preditores de forma sequencial de modo que cada preditor tenta se especializar (e corrigir) nos erros do classificador anterior. Embora existam muitos algoritmos de aprendizado ensemble que exploram a proposta de Boosting, o mais comum é o Adaboost. Neste algoritmo, cada instância possui um peso associado, os quais inicialmente são uniformes. A cada classificador treinado, o peso das instâncias classificadas corretamente é diminuído, enquanto o peso das instâncias classificadas incorretamente é aumentado. Mais informações podem ser obtidas na documentação do método [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). Os principais parâmetros deste método são:

*   base_estimator: o algoritmo usado para treinamento dos modelos individuais. Por padrão, utiliza o DecisionTreeClassifier com max_depth=1 (alto viés).
*   n_estimators: o número de modelos do ensemble. Por padrão, utiliza 50.
*   learning_rate: o peso atribuído a cada classificador em cada iteração do Boosting. Quanto maior o valor, mais aumenta a contribuição de cada classificador. Por padrão, utiliza learning_rate igual a 1. 


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

## Treina um classificador AdaBoost, com 100 modelos criados de forma sequencial
## Cada modelo é uma árvore de decisão com max_depth=1
adab_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
adab_clf.fit(X_train, y_train)

In [ ]:
y_pred_adab = adab_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred_adab,labels=adab_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=adab_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print(round(accuracy_score(y_test, y_pred_adab),3))

O sklearn possui uma versão de AdaBoost para regressão: [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor)


### Voting Classifier

Uma das formas mais simples de criar um modelo ensemble utilizando diferentes algoritmos de aprendizado de máquina é através do método de votação. O sklearn possui um método que facilita a aplicação desta estratégia: [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier). Os principais parâmetros do método são:


*   estimators: lista dos algoritmos a serem usados como base do ensemble
*   voting: 'hard' para aplicar uma votação majoritária (baseada nos rótulos preditos), ou 'soft' para retornar como saída do ensemble a classe que maximiza a soma das probabilidades preditas por todos os modelos. Por padrão, utiliza 'hard'.
*   weights: pesos a serem atribuídos a cada classificador. Por padrão, utiliza pesos uniformes.



Vamos exemplificar o seu uso a seguir.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)
voting_clf.fit(X_train, y_train)

In [ ]:
y_pred_voting = voting_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred_voting,labels=voting_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=voting_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print(round(accuracy_score(y_test, y_pred_voting),3))

### Stacking

Outra estratégia para a agregação de modelos treinados com diferentes métodos de aprendizado a partir dos mesmos dados é o Stacking. A ideia é simples: após o treinamento dos classificadores individuais (as 'bases' do ensemble), um modelo é treinado para combinar as saídas deles em uma única decisão. Esta estratégia está implementada no método [StackingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier). Os principais parâmetros do método são:

*   estimators: lista dos algoritmos a serem usados como base do ensemble
*   final_estimator: algoritmo a ser usado para combinar a saída dos classificadores bases. Por padrão, usa LogisticRegression
*   cv: determina a configuração da validação cruzada (método usado para treinamento e validação de modelos) a ser utilizada para treinar o final_estimator. Pode ser um valor inteiro que define número de folds, ou 'prefit' quando os classificadores base já estão treinados. Por padrão, usa 5-fold cross-validation



In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # number of cross-validation folds
)
stacking_clf.fit(X_train, y_train)

In [ ]:
y_pred_stacking = stacking_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred_stacking,labels=stacking_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=stacking_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print(round(accuracy_score(y_test, y_pred_stacking),3))

O sklearn possui uma versão de VotingClassifier para regressão: [VotingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html#sklearn.ensemble.VotingRegressor)
